**Notebook preparation and importing libraries**

In [1]:
import warnings
from hvac import Quantity
from hvac.fluids import HumidAir, Fluid, CoolPropWarning

In [2]:
warnings.filterwarnings('ignore', category=CoolPropWarning)

In [3]:
Q_ = Quantity

In [4]:
import hvac.heat_transfer.heat_exchanger.fin_tube.air_evaporator.rating as rating

# Rating of a plain fin-tube counterflow air evaporator

Define the type of refrigerant:

In [5]:
R134a = Fluid('R134a')

Define the type of evaporator: a plain fin-tube, counterflow heat exchanger.

In [6]:
Evaporator = rating.PFT_CO_EVP

Create an instance of the evaporator model, specifying the dimensions of the frontal area, the number of rows, the geometrical characteristics of the heat transfer surfaces, and the thermal conductivity of the fins:

In [7]:
evp = Evaporator(
    L1=Q_(0.731, 'm'),          # width of frontal area
    L3=Q_(0.244, 'm'),          # height of frontal area
    N_r=2,                      # number of rows
    S_t=Q_(25.4, 'mm'),         # vertical distance between tubes
    S_l=Q_(22.0, 'mm'),         # horizontal distance between tubes
    D_i=Q_(8.422, 'mm'),        # inner tube diameter
    D_o=Q_(10.2, 'mm'),         # outer tube diameter
    t_f=Q_(0.3302, 'mm'),       # fin thickness
    N_f=1 / Q_(3.175, 'mm'),    # fin density
    k_f=Q_(237, 'W / (m * K)')  # conductivity of fin material
)

On this instance, we set the known operating conditions:
- the state of air entering the evaporator
- the mass flow rate of air through the evaporator
- the state of saturated refrigerant liquid/vapor mixture entering the evaporator (either evaporation temperature or pressure, and vapor quality)
- the degree of superheat of the refrigerant at the evaporator outlet, which is a setting of the expansion device

The result of rating the evaporator will contain:
- the mass flow rate of refrigerant let through by the expansion device in order to reach the set degree of superheat
- the resulting state of air at the air-side evaporator outlet
- the heat absorption rate (the heat transfer rate from the air to the refrigerant)
- the air-side pressure drop
- the flow length used for superheating the refrigerant vapor to the set degree of superheat

The rating routine is an iterative routine which is started with an intial guess of the mass flow rate of refrigerant. This initial guess may be the mass flow rate that was determined in the preliminary heat exchanger calculations. The criterion to stop the iterative routine is the tolerated deviation (tolerance) between the last and the previous calculated mass flow rate of refrigerant, which can be set through parameter `tol_m_dot_rfg`. Should the tolerance not be reached after a maximum number of iterations, which can be set with parameter `i_max`, a `ValueError` exception will be raised and no result is returned.

Suppose evaporation temperature is 5 °C, condensation temperature is 50 °C, and the degree of subcooling of the refrigerant leaving the condenser is 5 K. In that case the state of refrigerant entering the evaporator can be determined as:

In [8]:
cnd_rfg_sat_liq = R134a(T=Q_(50, 'degC'), x=Q_(0, 'frac'))
P_cnd = cnd_rfg_sat_liq.P
dT_sc = Q_(5, 'K')
cnd_rfg_out = R134a(T=cnd_rfg_sat_liq.T.to('K') - dT_sc, P=P_cnd)
evp_rfg_sat_vap = R134a(T=Q_(5, 'degC'), x=Q_(1, 'frac'))
P_evp = evp_rfg_sat_vap.P
evp_rfg_in = R134a(P=P_evp, h=cnd_rfg_out.h)

Now we set the operating conditions on the evaporator model and get its state of performance:

In [9]:
r = evp(
    air_in=HumidAir(Tdb=Q_(24, 'degC'), RH=Q_(50, 'pct')),
    m_dot_air=Q_(1500, 'kg / hr'),
    rfg_in=evp_rfg_in,
    dT_rfg_sh=Q_(10, 'K'),
    m_dot_rfg_ini=Q_(114.344, 'kg / hr'),
    tol_m_dot_rfg=Q_(0.01, 'kg / hr')
)

Showing the results:

In [10]:
print(
    f"m_dot_rfg = {r.m_dot_rfg.to('kg / hr'):~P.3f}\n"
    f"air_out: {r.air_out.Tdb.to('degC'):~P.1f} DB, "
    f"{r.air_out.RH.to('pct'):~P.0f} RH\n"
    f"Q = {r.Q_dot.to('kW'):~P.3f}\n"
    f"dP_air = {r.dP_air.to('Pa'):~P.0f}\n"
    f"superheating flow length = {r.L2_superheat.to('mm'):~P.0f}"
)

m_dot_rfg = 100.128 kg/h
air_out: 17.5 °C DB, 65 % RH
Q = 4.081 kW
dP_air = 14 Pa
superheating flow length = 13 mm


We can now compare these results with our preliminary calculations. By increasing or decreasing the number of rows of the evaporator, we can try to get at the desired state of air leaving the evaporator (or we can try with other heat exchanger core geometries also).